In [39]:
import duckdb
from pathlib import Path
import pandas as pd

Task 1 - EDA in python

In [ ]:
# duckdb database path
duckdb_path = "data/sakila.duckdb"      # sets the path for the DuckDB database file
Path(duckdb_path).unlink(missing_ok=True)  # deletes the duckdb file if it already exists

# Ingest Sakila dataset into DuckDB
with duckdb.connect(duckdb_path) as conn, open("sql/load_sakila.sql") as ingest_scripts:  # opens a Duckdb connection and the SQL script file
    conn.sql(ingest_scripts.read()) # 

    films = conn.sql("FROM film;").df()

films

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,A Unbelieveable Yarn of a Boat And a Database ...,2006,1,<NA>,6,0.99,183,9.99,G,"Trailers,Behind the Scenes",2021-03-06 15:52:08
996,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,2006,1,<NA>,4,0.99,179,14.99,NC-17,"Trailers,Behind the Scenes",2021-03-06 15:52:08
997,998,ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...,2006,1,<NA>,6,0.99,105,10.99,NC-17,Deleted Scenes,2021-03-06 15:52:08
998,999,ZOOLANDER FICTION,A Fateful Reflection of a Waitress And a Boat ...,2006,1,<NA>,5,2.99,101,28.99,R,"Trailers,Deleted Scenes",2021-03-06 15:52:08


In [ ]:
description

In [ ]:
# Load all tables into pandas DataFrames
dfs = {}
with duckdb.connect(duckdb_path) as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"SELECT * FROM {name};").df()

dfs.keys()

dict_keys(['actor', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store'])

In [62]:
dfs ["film_list"].head()

,FID,title,description,category,price,length,rating,actors
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,Documentary,0.99,86,PG,PENELOPE GUINESS
1,23,ANACONDA CONFESSIONS,A Lacklusture Display of a Dentist And a Denti...,Animation,0.99,92,R,PENELOPE GUINESS
2,25,ANGELS LIFE,A Thoughtful Display of a Woman And a Astronau...,New,2.99,74,G,PENELOPE GUINESS
3,106,BULWORTH COMMANDMENTS,A Amazing Display of a Mad Cow And a Pioneer w...,Games,2.99,61,G,PENELOPE GUINESS
4,140,CHEAPER CLYDE,A Emotional Character Study of a Pioneer And a...,Sci-Fi,0.99,87,G,PENELOPE GUINESS


a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [ ]:
# Count the number of films in the film table
film = duckdb.sql("""
    SELECT title, length
    FROM film
    WHERE length > 180
""").df()

film.head() 

,title,length
0,ANALYZE HOOSIERS,181
1,BAKED CLEOPATRA,182
2,CATCH AMISTAD,183
3,CHICAGO NORTH,185
4,CONSPIRACY SPIRIT,184


In [ ]:
# Count the number of films in the film table
duckdb.sql( """SELECT * FROM film """).df()

,title,length
0,ANALYZE HOOSIERS,181
1,BAKED CLEOPATRA,182
2,CATCH AMISTAD,183
3,CHICAGO NORTH,185
4,CONSPIRACY SPIRIT,184
5,CONTROL ANTHEM,185
6,CRYSTAL BREAKING,184
7,DARN FORRESTER,185
8,FRONTIER CABIN,183
9,GANGS PRIDE,185


In [ ]:
# Count the number of films in the film table
duckdb.sql( "SELECT count (*) FROM film ").df()

,count_star()
0,39
